Name: Patrick Ng  
Class: W261-2  
Date: Mar 19, 2016  
HW09

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

## HW 9.0: Short answer questions

What is PageRank and what is it used for in the context of web search?  
What modifications have to be made to the webgraph in order to leverage the machinery of Markov Chains to 
compute the steady state distibuton?  


#### What is PageRank and what is it used for in the context of web search?  
PageRank is a link analysis algorithm and it assigns a numerical weighting to each element of a hyperlinked set of documents, such as the World Wide Web, with the purpose of "measuring" its relative importance within the set.  In the context of web search, it is used for ranking the documents in the posting list of the inverted index, so that relatively more important documents will be put at the beginning of the posting list.  
  
#### What modifications have to be made to the webgraph in order to leverage the machinery of Markov Chains to compute the steady state distibuton?  
In order to leverage the machinery of Markov Chains to compute the steady state distibuton, we need to make two adjustments to the webgraph (i.e. the transition matrix H, where each row represents a node):
+ **Stochasticity adjustment** to resolve dangling nodes (nodes with no outbound links) problem:
    + For each zero row we replace each element with 1/n, where n is the number of nodes in the wegraph.
+ **Primitivity adjustment** to guarantee convergence.
    + We need to define a teleport probability $\alpha$ and define the transition matrix as:  
<br/>
\begin{equation} 
P = (1-\alpha)H + {\alpha}I(1/n)  
\end{equation}  
where:  
        - _H_ is the hyperlink matrix
        - n is the number of nodes in the webgraph
        - I is the identity matrix
        
#### OPTIONAL: In topic-specific pagerank, how can we insure that the irreducible property is satified? (HINT: see HW9.4)  


## HW 9.1: MRJob implementation of basic PageRank

Write a basic MRJob implementation of the iterative PageRank algorithm
that takes sparse adjacency lists as input (as explored in HW 7).
Make sure that you implementation utilizes teleportation (1-damping/the number of nodes in the network), 
and further, distributes the mass of dangling nodes with each iteration
so that the output of each iteration is correctly normalized (sums to 1).  

NOTE: 
+ The PageRank algorithm assumes that a random surfer (walker), starting from a random web page,
chooses the next page to which it will move by clicking at random, with probability d,
one of the hyperlinks in the current page. This probability is represented by a so-called
‘damping factor’ d, where d ∈ (0, 1). Otherwise, with probability (1 − d), the surfer
jumps to any web page in the network. If a page is a dangling end, meaning it has no
outgoing hyperlinks, the random surfer selects an arbitrary web page from a uniform
distribution and “teleports” to that page]


As you build your code, use the test data

s3://ucb-mids-mls-networks/PageRank-test.txt
Or under the Data Subfolder for HW7 on Dropbox with the same file name.  
(On Dropbox https://www.dropbox.com/sh/2c0k5adwz36lkcw/AAAAKsjQfF9uHfv-X9mCqr9wa?dl=0)

with teleportation parameter set to 0.15 (1-d, where d, the damping factor is set to 0.85), and crosscheck
your work with the true result, displayed in the first image
in the Wikipedia article:

https://en.wikipedia.org/wiki/PageRank

and here for reference are the corresponding PageRank probabilities:
```
A,0.033
B,0.384
C,0.343
D,0.039
E,0.081
F,0.039
G,0.016
H,0.016
I,0.016
J,0.016
K,0.016
```

### Initialize the graph structure based on the adjaceny list file

In [30]:
%%writefile MrInitGraph.py
import re
from mrjob.job import MRJob, MRStep
import mrjob
import json
import sys

class MrInitGraph(MRJob):

    SORT_VALUES = True  # Need 2nd sort
   
    def mapper(self, _, line):
        # input format:
        # Json object:
        # 1       {'2': 1, '6': 1}
        
        fields = line.strip().split('\t')
        node = fields[0]

        # JSON uses double quote for string
        adjList = json.loads(fields[1].replace("'", '"'))
        
        # Output for the node, which is an outbound node
        yield node, [0, adjList] # 0 is used for 2nd sorting
        
        # Also need to output an empty entry for every nodes in the adjList.
        # It is needed for directed graph, so that in the reducer we can generate 
        # an entry for nodes which don't have an outbound link.
        for key in adjList.keys():
            yield key, [1]  # 1 is used for 2nd sorting
            
    def reducer(self, node, values):
        value = values.next()
        
        # An outbound node will be seen first because we use 2nd sorting
        if value[0] == 0:
            # It's an outbound record.  
            # Just yield the node without processing the rest of values.
            yield node, [value[1:], 0]
        else:
            # In a directed graph, if a node has no outbound link, generate an entry for it.
            # And ignore the rest of values
            yield node, [{}, 0]
                                                              
if __name__ == '__main__':
    MrInitGraph.run()

Overwriting MrInitGraph.py


### Calculate the number of nodes

In [31]:
%%writefile MrFindNumberOfNodes.py
import re
from mrjob.job import MRJob, MRStep
import mrjob
import json
import sys

class MrFindNumberOfNodes(MRJob):
    
    def steps(self):
        return [MRStep(mapper_init=self.mapper_init, mapper=self.mapper, mapper_final = self.mapper_final,
                   combiner = self.reducer,
                   reducer=self.reducer
            )]

    def mapper_init(self):
        self.partial_size = 0
        
    def mapper(self, _, line):
        self.partial_size += 1
        
    def mapper_final(self):
        yield None, self.partial_size
        
    def reducer(self, key, values):
        yield None, sum(values)
                                                              
if __name__ == '__main__':
    MrFindNumberOfNodes.run()

Overwriting MrFindNumberOfNodes.py


### A PageRank iteration

In [34]:
%%writefile MrPageRankIteration.py
import re
from mrjob.job import MRJob, MRStep
import mrjob
import json
import sys

class MrPageRankIteration(MRJob):

    INPUT_PROTOCOL = mrjob.protocol.JSONProtocol
    
    Type_Graph = 0
    Type_PR = 1
    
    Key_LostPR = "#"
    
    def configure_options(self):
        super(MrInitPageRank, self).configure_options()
        self.add_passthrough_option(
            '--initWithGraphSize', type='int', default=None)

    def mapper_init(self):
        self.lostPR = 0
        
    def mapper(self, node, data):
        adjList, pageRank = data
        
        # We're at the first iteration, and PageRank value hasn't been initialized yet.
        if self.options.initWithGraphSize is not None:
            pageRank = 1 / self.options.initWithGraphSize

        # Pass along the graph structure
        yield node, [self.Type_Graph, adjList]
        
        if len(adjList) > 0:
            # The PR juice we will send out
            p = pageRank / len(adjList)
            for link in adjList.keys():
                yield link, [self.Type_PR, p] # 1 means it's a PR contribution
        else:
            self.lostPR += pageRank
            
    def mapper_final(self):
        yield self.Key_LostPR, self.lostPR

    def reducer(self, node, values):
        pageRank = 0
        for value in values:
            valueType, data = value
            if valueType == self.Type_Graph:
                adjList = data
            else:
                pageRank += float(data)

        # Note: we haven't handled lostPR yet!!!!!!
        yield node, [adjList, pageRank]
            
                                                              
if __name__ == '__main__':
    MrPageRankIteration.run()

Writing MrPageRankIteration.py


### Driver for HW91

In [32]:
%%writefile Driver_Hw91.py
from MrFindNumberOfNodes import MrFindNumberOfNodes
from MrInitGraph import MrInitGraph
import time

import argparse
parser = argparse.ArgumentParser()
parser.add_argument("--inputFile", type=str)
parser.add_argument("--ec2-instance-type", type=str, default=None)
parser.add_argument("--initGraphFolder", type=str, default=None)
parser.add_argument("--num-ec2-instances", type=str, default=None)
parser.add_argument("-r", "--run", type=str)
args = parser.parse_args()

jobArgsBase = ['-r', args.run,
             '--no-strict-protocols']

if args.run == "emr":
    jobArgsBase += ["--pool-emr-job-flows"]
    
    if args.ec2_instance_type is not None:
        jobArgsBase += ["--ec2-instance-type", args.ec2_instance_type]
        
    if args.num_ec2_instances is not None:
        jobArgsBase += ["--num-ec2-instances", args.num_ec2_instances]

i = 1
startTime = time.time()

def ensureFolderPath(path):
    return path if path.endswith("/") else path + "/"

# Initialize the graph structure
mr_jobInitGraph = MrInitGraph(args=[args.inputFile] + jobArgsBase)
with mr_jobInitGraph.make_runner() as runner: 
    runner.run()

    if args.run == "inline" or args.run == "local":
        initGraphFile = "_initGraph.txt"
    else:
        initGraphFile = ensureFolderPath(args.initGraphFolder)
        jobArgs += ["--no-output", "--output-dir", interSsspFile]

    if args.run == "inline" or args.run == "local":
        # Generate the initGraph file
        with open(initGraphFile, 'w') as f:            
            for line in runner.stream_output():
                f.write("%s" % (line))
    
# Find the number of nodes
mr_jobNumberOfNodes = MrFindNumberOfNodes(args=[initGraphFile] + jobArgsBase)
with mr_jobNumberOfNodes.make_runner() as runner: 
    runner.run()

    for line in runner.stream_output():
        key, value =  mr_jobNumberOfNodes.parse_output_line(line)
        nodeCount = int(value)
    
    print "Number of nodes = %d" % (nodeCount)

print
print "Total time: %d sec" % (time.time() - startTime)

Overwriting Driver_Hw91.py


In [33]:
!python Driver_Hw91.py -r inline --inputFile PageRank-test.txt

No handlers could be found for logger "mrjob.sim"
Number of nodes = 11

Total time: 0 sec
